# Serverless HDF Database

HDF files can be considered databases. What is missing are query functions. This chapter will take you through everything you need to know to find data within one or more HDF5 files based on attributes.

In [1]:
import h5rdmtoolbox as h5tbx
from h5rdmtoolbox.database import hdfdb
from h5rdmtoolbox import tutorial

h5tbx.use(None)

using("h5py")

## Searching within a single file

Imagine a large, nested HDF5 file containing data from a fluid experiment. It contains many datasets, some represent pressure data, others are velocity datasets. With future reuse in mind, we assigned the attribute "standard_name" to the datasets during data creation. We now want ... 

 ... **to find all datasets with attribute "standard_name" equal to "velocity"**

The example file is provided by the tutorial module:

In [2]:
filename = tutorial.generate_fluid_hdf_file()
h5tbx.dump(filename)

The database we are going to use is the `GroupDB`. It takes an opened HDF5 group as input. Then we can call either `find` or `find_one` on the database object. Since we are looking for *all* velocity datasets, we will choose `find`:

In [3]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    res = db.find({'standard_name': 'velocity'})

    for r in res:
        print(r.name)

/group1/velocity
/velocity
/group2/velocity


**So how does the query work?**

Note, that the input for the `find()` and `find_one()` methods expect arguments similar to the `pymongo` syntax. If you are familiar to it, usage will be very natural for you. If not, here's how the filter request is built up:

The methods expect a dictionary in the following way:

  `find({<kw_pr_prop1>: <value_or_operator1>, <kw_pr_prop1>: <value_or_operator2>, ...}, <object_filter>)`

The first argument is the main filter argument. It is a dictionary, containing one or multiple keyword-value-pairs:
 - The simplest filter argument is the one we used above, which searches for the exact match of an attribute (in our case "standard_name") and its value (in our case "velocity").
 - To match a HDF5 dataset or group property, like the HDF5 internal path (`.name`) use "\\$name" for the keyword (mind the dollar sign!)
 - Instead of the value, we could also provide an "operator", e.g. {"\\$eq": "velocity"}. An operator always starts with a "\\$"-sign. The above function called would look like this:<br>`db.find({'standard_name': {'$eq': 'velocity'}})`
 - We could have searched for HDF5 objects, which require multiple attribute key-value pairs to match. E.g. we could have queried for the units attribute, too:<br>
 `db.find({'standard_name': 'velocity', 'units': 'm/s'})`

With the *second* argument (*<object_filter>*) we can select, whether we want to search for datasets or groups only, or for both (default). In our query, we were looking for datasets only, so a better query would have been the following:<br>
`db.find({'standard_name': 'velocity', 'units': 'm/s'}, 'dataset')`

## Advanced queries

Sometimes, we want to perform more sophisticated queries, e.g. we would like to ... 
1. ... find all datasets that have a certain attribute key (the value is not of interest),
2. ... find attributes where the string matches a regex expression,
3. ... find attributes with numeric values that are greater than, lower than, ... a certain value,
4. ... find attributes, which match our own operator

The following subsections show how it is done:

### 1. Operator: Exists:

Find objects which have the attribute "units"

In [4]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'units': {'$exists': True}})
    for r in results:
        print(r.name)

/pressure1
/velocity
/group2/velocity
/group2/pressure2
/group1/velocity
/group2/z


Find objects which *don't* have the attribute "units"

In [5]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'units': {'$exists': False}}, recursive=True)
    for r in results:
        print(r.name)

/group1
/group2


### 2. Operator: Regex

Let's find all datasets, which's names (internal HDF path) end with "/velocity". In this example, we are not matching an attribute name but a **property**. To do so, place a dollar sign in front:

In [6]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'$name': {'$regex': '^.*/velocity$'}}, 'dataset')
    for r in results:
        print(r.name)

/group2/velocity
/velocity
/group1/velocity


### 3. Mathematical operation (>, >=, <, <=)

The datasets and groups in the example files all have the attribute "check_value", let's identify objects that match certain values of this attribute:

In [7]:
print('object names with check value greater than 0.1:')

with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'check_value': {'$gt': 0.1}})
    for r in results:
        print('  ', r.name)


print('\nobject names with check value equal to 0:')

with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'check_value': {'$eq': 0}})
    for r in results:
        print('  ', r.name)


print('\nobject names with check value lower equal -3.3:')

with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'check_value': {'$lte': -3.3}})
    for r in results:
        print('  ', r.name)

object names with check value greater than 0.1:
   /velocity
   /group2/velocity

object names with check value equal to 0:
   /group1
   /group2
   /

object names with check value lower equal -3.3:
   /pressure1
   /group2/pressure2


Note, that besides "name", we can also match HDF5 **any other object property**:

In [27]:
with h5tbx.File(filename) as h5:
    print('Ndim == 1:')
    
    db = hdfdb.GroupDB(h5)
    results = db.find({'$ndim': {'$eq': 1}}, 'dataset')
    for r in results:
        print('  ', r.name)

    print('\nNdim > 2:')
    db = hdfdb.GroupDB(h5)
    results = db.find({'$ndim': {'$gt': 2}}, 'dataset')
    for r in results:
        print('  ', r.name)

Ndim == 1:
   /pressure1
   /velocity
   /group2/velocity
   /group2/pressure2
   /group1/velocity

Ndim > 2:


### 4. User-defined operator

Let's take the regex example and turn it into a new operator:

The regex query filter `"{'$name': {'$regex': '^.*/velocity$'}}"` means, that we are looking for "basenames" of objects, so not the full internal HDF5 path but the name.

All operator functions are stored in `database.hdfdb.query`. In fact, the "\\$basename"-operator exists, so we first delete it and add it again by our own function:

In [8]:
from h5rdmtoolbox.database.hdfdb import query

# note, all operator functions are stored in this dictionary: query.operator
query.operator.pop('$basename', None)  # remove existing operator

def my_basename_operator(obj_name, basename) -> bool:
    """calling regex under the hood"""
    print(f'Checking if basename of object name "{obj_name}" is matching pattern "^.*/{basename}$"')
    return query._regex(obj_name, pattern=f'^.*/{basename}$')

query.operator['$basename'] = my_basename_operator

In [9]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find({'$name': {'$basename': 'velocity'}}, 'dataset')

    for r in results:
        print(r.name)

Checking if basename of object name "/group1/velocity" is matching pattern "^.*/velocity$"
Checking if basename of object name "/group2/pressure2" is matching pattern "^.*/velocity$"
Checking if basename of object name "/group2/velocity" is matching pattern "^.*/velocity$"
Checking if basename of object name "/group2/z" is matching pattern "^.*/velocity$"
Checking if basename of object name "/pressure1" is matching pattern "^.*/velocity$"
Checking if basename of object name "/velocity" is matching pattern "^.*/velocity$"
/group1/velocity
/velocity
/group2/velocity


# Working with results

Let's investigate the return values of queries. The method `find_one` returns a so-called "lazy" object, and the `find` method a generator of this class.

In [18]:
with h5tbx.File(filename) as h5:
    db = hdfdb.GroupDB(h5)
    results = db.find_one({'units': {'$exists': True}}, 'dataset')
results

<LDataset "/group1/velocity" in "C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp_0\tmp0.hdf" attrs=(standard_name=velocity, units=m/s)>

Lazy objects are datasets or groups which are accessible even if the file is closed. This does not mean that the data is in the RAM, but that the data of is accessed only when needed (the file is opened in the background):

In [21]:
results[()]  # file is opened in the background

<xarray.DataArray 'velocity' (dim_0: 5)>
4 0 -3 12 3
Dimensions without coordinates: dim_0
Attributes:
    standard_name:  velocity
    units:          m/s